In [1]:
import sys
sys.path.insert(0, '../')

In [2]:
from typing import Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Subset
from torchvision import datasets

import albumentations as A
import albumentations.pytorch.transforms as AT

import tensorclan
import tensorclan.dataset as tcd
import tensorclan.model as tcm
import tensorclan.dataset as tcd
import tensorclan.dataset.augmentation as tca
import tensorclan.utils as tc_utils
import tensorclan.runner as tc_runner

In [3]:
%%writefile custom_model.yaml

name: TTC_EXAMPLE
log_dir: logs
chkpt_dir: checkpoint
use_checkpoints: false

device: GPU

model:
    name: Net

dataset:
    name: MNISTV2
    transforms: MNISTTransformsV2
    root: data
    loader_args:
        batch_size: 128
        num_workers: 2
        shuffle: True
        pin_memory: True

loss: CrossEntropyLoss

optimizer:
    type: SGD
    args:
        lr: 0.01
        momentum: 0.95

training:
    epochs: 5

Overwriting custom_model.yaml


In [4]:
@tcm.model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

@tcd.dataset
class MNISTV2(tcd.BaseDataset):
    test_set: datasets.MNIST
    train_set: datasets.MNIST

    def __init__(self, root: str, transforms: tca.BaseAugmentation):
        self.data_dir = root

        self.train_set = datasets.MNIST(
            self.data_dir,
            train=True,
            download=True,
            transform=transforms.build_transforms(train=True),
        )

        self.test_set = datasets.MNIST(
            self.data_dir,
            train=False,
            download=True,
            transform=transforms.build_transforms(train=False)
        )

    def split_dataset(self):
        return Subset(self.train_set, indices=range(0, len(self.train_set))), Subset(self.test_set, indices=range(0, len(self.test_set)))

    @staticmethod
    def plot_sample(sample):
        pass

@tca.augmentation
class MNISTTransformsV2(tca.BaseAugmentation):

    mean: Tuple[float] = (0.1307,)
    std: Tuple[float] = (0.3081,)

    def build_train(self):
        train_transforms = A.Compose([
            A.Normalize(mean=self.mean, std=self.std),
            AT.ToTensor()
        ])
        return train_transforms

    def build_test(self):
        test_transforms = A.Compose([
            A.Normalize(mean=self.mean, std=self.std),
            AT.ToTensor()
        ])

        return test_transforms

[ 2020-07-31 12:24:18,176 - tensorclan.model.model ] INFO: Registered Net as model
[ 2020-07-31 12:24:18,178 - tensorclan.dataset.dataset ] INFO: Registered MNISTV2 as dataset
[ 2020-07-31 12:24:18,179 - tensorclan.dataset.augmentation.augmentation ] INFO: Registered MNISTTransformsV2 as augmentation


In [5]:
config = tc_utils.load_config('custom_model.yaml')

In [6]:
runner = tc_runner.Runner(config=config)


[ 2020-07-31 12:24:20,971 - tensorclan.runner.runner ] INFO: => Now simply setup_train and then start_train your model


In [7]:
runner.setup_train()

[ 2020-07-31 12:24:22,659 - tensorclan.runner.runner ] INFO: => Config
[ 2020-07-31 12:24:22,663 - tensorclan.runner.runner ] INFO: {'chkpt_dir': 'checkpoint',
[ 2020-07-31 12:24:22,665 - tensorclan.runner.runner ] INFO:  'dataset': {'loader_args': {'batch_size': 128,
[ 2020-07-31 12:24:22,667 - tensorclan.runner.runner ] INFO:                              'num_workers': 2,
[ 2020-07-31 12:24:22,669 - tensorclan.runner.runner ] INFO:                              'pin_memory': True,
[ 2020-07-31 12:24:22,671 - tensorclan.runner.runner ] INFO:                              'shuffle': True},
[ 2020-07-31 12:24:22,673 - tensorclan.runner.runner ] INFO:              'name': 'MNISTV2',
[ 2020-07-31 12:24:22,674 - tensorclan.runner.runner ] INFO:              'root': 'data',
[ 2020-07-31 12:24:22,675 - tensorclan.runner.runner ] INFO:              'transforms': 'MNISTTransformsV2'},
[ 2020-07-31 12:24:22,677 - tensorclan.runner.runner ] INFO:  'device': 'GPU',
[ 2020-07-31 12:24:22,678 - tenso

In [8]:
runner.start_train()

[ 2020-07-31 12:24:31,615 - tensorclan.trainer.gpu_trainer ] INFO: => Training Started
[ 2020-07-31 12:24:31,617 - tensorclan.trainer.gpu_trainer ] INFO: Training the model for 5 epochs
[ 2020-07-31 12:24:31,619 - tensorclan.trainer.gpu_trainer ] INFO: => Training Epoch 0
[ 2020-07-31 12:24:35,707 - tensorclan.trainer.gpu_trainer ] INFO: loss: 0.6630898202215431, accuracy: 78.05666666666667
[ 2020-07-31 12:24:35,708 - tensorclan.trainer.gpu_trainer ] INFO: => Testing Epoch 0
[ 2020-07-31 12:24:36,446 - tensorclan.trainer.gpu_trainer ] INFO: loss: 0.10670972257098064 accuracy: 96.72
[ 2020-07-31 12:24:36,448 - tensorclan.trainer.gpu_trainer ] INFO: 

[ 2020-07-31 12:24:36,449 - tensorclan.trainer.gpu_trainer ] INFO: => Training Epoch 1
[ 2020-07-31 12:24:40,388 - tensorclan.trainer.gpu_trainer ] INFO: loss: 0.2715205023251871, accuracy: 91.99333333333334
[ 2020-07-31 12:24:40,389 - tensorclan.trainer.gpu_trainer ] INFO: => Testing Epoch 1
[ 2020-07-31 12:24:41,125 - tensorclan.trainer.g